In [9]:
import re
import json
import csv
import requests
from bs4 import BeautifulSoup ,SoupStrainer
import pandas as pd 
from io import StringIO
from pathlib import Path  

#!pip install selenium

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service



import sys

import unittest

import time


In [10]:
def get_infinity_page_html_data(chromedriver_path="./chromedriver.exe", scroll_number=50, sleep_time=2, url = ""):
    # Create selenium driver
    s = Service(chromedriver_path)
    driver = webdriver.Chrome(service=s)
    driver.get(url)

    for i in range(1,scroll_number):
        print(f"scroll_number: {i}")
        driver.execute_script("window.scrollTo(1,100000)")
        time.sleep(sleep_time)
    html_data = driver.page_source
    driver.close()
    
    soup = BeautifulSoup(html_data, 'html.parser')    
    return soup


In [11]:
def make_table(soup):
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    return df

In [12]:
def data_cleaning(df):
    df = df[df["Open"].str.contains("Dividend") == False]
    df = df[df["Open"].str.contains("Stock") == False]
    df.drop(df.tail(1).index,inplace=True)
    return df

In [13]:
def write_to_file(df, path):
    from pathlib import Path
    filepath = Path(path)
    filepath.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(filepath) 

In [15]:
base_url = "https://finance.yahoo.com/quote/"
period_url = "/history?period1=1262304000&period2=1656115200&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
indexes = ["NVDA","AAPL","TSLA","AMD","MSFT","INTC"]
main_df = pd.DataFrame()

for index in indexes:
    soup = get_infinity_page_html_data(url = (base_url + index + period_url))
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    
    df = data_cleaning(df)
    
    path = index + ".csv"
    write_to_file(df, path)
    

scroll_number: 1
scroll_number: 2
scroll_number: 3
scroll_number: 4
scroll_number: 5
scroll_number: 6
scroll_number: 7
scroll_number: 8
scroll_number: 9
scroll_number: 10
scroll_number: 11
scroll_number: 12
scroll_number: 13
scroll_number: 14
scroll_number: 15
scroll_number: 16
scroll_number: 17
scroll_number: 18
scroll_number: 19
scroll_number: 20
scroll_number: 21
scroll_number: 22
scroll_number: 23
scroll_number: 24
scroll_number: 25
scroll_number: 26
scroll_number: 27
scroll_number: 28
scroll_number: 29
scroll_number: 30
scroll_number: 31
scroll_number: 32
scroll_number: 33
scroll_number: 34
scroll_number: 35
scroll_number: 36
scroll_number: 37
scroll_number: 38
scroll_number: 39
scroll_number: 40
scroll_number: 41
scroll_number: 42
scroll_number: 43
scroll_number: 44
scroll_number: 45
scroll_number: 46
scroll_number: 47
scroll_number: 48
scroll_number: 49
scroll_number: 1
scroll_number: 2
scroll_number: 3
scroll_number: 4
scroll_number: 5
scroll_number: 6
scroll_number: 7
scroll_n

In [16]:
indexes = ["NVDA","AAPL","TSLA","AMD","MSFT","INTC"]
main_df = pd.DataFrame()

for index in indexes:
    
    df = pd.read_csv( index + '.csv')
    df.drop(df.index[2500:], inplace=True)
    df.set_index('Date')
   
    
    main_df['Date'] = df['Date']
    main_df[index + " " + "Open"] = df['Open']
    main_df[index + " " + "High"] = df['High']
    main_df[index + " " + "Low"] = df['Low']
    main_df[index + " " + "Close"] = df['Close*']
    main_df[index + " " + "Volume"] = df['Volume']
    
    main_df[index + " " +'Daily change'] = (((df['Close*']).astype(float))-((df['Open']).astype(float)))/((df['Open']).astype(float))*100
    main_df[index + " " +'Daily change Max'] =  abs((df['High'].astype(float)-df['Low'].astype(float))/df['Low'].astype(float)*100)
    


    
    print(df)
    print(df.info())


main_df['Date'] = df['Date']
main_df.set_index('Date')
write_to_file(main_df, ("main_dataframe" + ".csv"))
main_df

      Unnamed: 0          Date    Open    High     Low  Close*  Adj Close**  \
0              0  Jun 24, 2022  165.00  171.40  163.10  171.26       171.26   
1              1  Jun 23, 2022  165.19  165.85  158.53  162.25       162.25   
2              2  Jun 22, 2022  162.26  166.62  161.80  163.60       163.60   
3              3  Jun 21, 2022  164.75  170.08  164.07  165.66       165.66   
4              4  Jun 17, 2022  156.48  159.95  153.28  158.80       158.80   
...          ...           ...     ...     ...     ...     ...          ...   
2495        2535  Jul 25, 2012    3.20    3.31    3.18    3.27         3.01   
2496        2536  Jul 24, 2012    3.25    3.27    3.17    3.21         2.95   
2497        2537  Jul 23, 2012    3.12    3.27    3.08    3.24         2.98   
2498        2538  Jul 20, 2012    3.28    3.30    3.17    3.20         2.94   
2499        2539  Jul 19, 2012    3.29    3.34    3.27    3.30         3.03   

        Volume  
0     47166300  
1     46368000  


,Date,NVDA Open,NVDA High,NVDA Low,NVDA Close,NVDA Volume,NVDA Daily change,NVDA Daily change Max,AAPL Open,AAPL High,...,MSFT Volume,MSFT Daily change,MSFT Daily change Max,INTC Open,INTC High,INTC Low,INTC Close,INTC Volume,INTC Daily change,INTC Daily change Max
0,"Jun 24, 2022",165.00,171.40,163.10,171.26,47166300,3.793939,5.088903,139.90,141.91,...,33900700,2.249723,2.391869,37.85,38.64,37.74,38.61,38152100,2.007926,2.384738
1,"Jun 23, 2022",165.19,165.85,158.53,162.25,46368000,-1.779769,4.617423,136.82,138.59,...,25861400,1.287319,2.263139,37.61,37.62,36.91,37.41,30163000,-0.531773,1.923598
2,"Jun 22, 2022",162.26,166.62,161.80,163.60,43713500,0.825835,2.978986,134.79,137.76,...,25939900,0.492278,2.715980,37.33,37.77,37.22,37.38,32571000,0.133941,1.477700
3,"Jun 21, 2022",164.75,170.08,164.07,165.66,48308900,0.552352,3.663071,133.42,137.06,...,29928300,1.390554,2.100116,37.36,38.03,37.33,37.73,34004900,0.990364,1.875167
4,"Jun 17, 2022",156.48,159.95,153.28,158.80,62905700,1.482618,4.351514,130.07,133.08,...,42800400,1.205558,2.651313,37.48,38.12,36.60,36.97,71235800,-1.360726,4.153005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,"Jul 25, 2012",3.20,3.31,3.18,3.27,41504400,2.187500,4.088050,20.52,20.74,...,45579500,-1.402189,1.911049,25.02,25.53,25.00,25.13,34407600,0.439648,2.120000
2496,"Jul 24, 2012",3.25,3.27,3.17,3.21,35116400,-1.230769,3.154574,21.69,21.77,...,47723300,-0.307798,1.591696,25.19,25.22,24.79,25.01,33637600,-0.714569,1.734570
2497,"Jul 23, 2012",3.12,3.27,3.08,3.24,45158400,3.846154,6.168831,21.23,21.64,...,55151900,-0.980724,1.964840,25.05,25.36,24.80,25.26,36378200,0.838323,2.258065
2498,"Jul 20, 2012",3.28,3.30,3.17,3.20,45100000,-2.439024,4.100946,21.89,21.94,...,64021700,-2.838710,3.327787,25.87,26.00,25.50,25.52,48844000,-1.352918,1.960784


In [17]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 43 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   2500 non-null   object 
 1   NVDA Open              2500 non-null   float64
 2   NVDA High              2500 non-null   float64
 3   NVDA Low               2500 non-null   float64
 4   NVDA Close             2500 non-null   float64
 5   NVDA Volume            2500 non-null   int64  
 6   NVDA Daily change      2500 non-null   float64
 7   NVDA Daily change Max  2500 non-null   float64
 8   AAPL Open              2500 non-null   float64
 9   AAPL High              2500 non-null   float64
 10  AAPL Low               2500 non-null   float64
 11  AAPL Close             2500 non-null   float64
 12  AAPL Volume            2500 non-null   int64  
 13  AAPL Daily change      2500 non-null   float64
 14  AAPL Daily change Max  2500 non-null   float64
 15  TSLA

In [18]:
tdf = pd.DataFrame(main_df, columns = ['NVDA Open','AAPL Open','AMD Open','MSFT Open','INTC Open','TSLA Open'])

In [19]:
tdf.tail()

,NVDA Open,AAPL Open,AMD Open,MSFT Open,INTC Open,TSLA Open
2495,3.20,20.52,4.11,29.24,25.02,5.98
2496,3.25,21.69,4.18,29.24,25.19,6.13
2497,3.12,21.23,4.15,29.57,25.05,6.21
2498,3.28,21.89,4.62,31.00,25.87,6.41
2499,3.29,21.83,4.93,30.51,26.35,6.54


In [20]:
indexes = ["NVDA","AAPL","TSLA","AMD","MSFT","INTC"]

for index in indexes:
      
    main_df[index + " " +'Daily change'] = (((main_df[index + " " +'Close']).astype(float))-((main_df[index + " " +'Open']).astype(float)))/((main_df[index + " " +'Open']).astype(float))*100
    main_df[index + " " +'Daily change Max'] =  abs((main_df[index + " " +'High'].astype(float)-main_df[index + " " +'Low'].astype(float))/main_df[index + " " +'Low'].astype(float)*100)


write_to_file(main_df, ("main_dataframe" + ".csv"))


In [21]:
main_df.tail()

,Date,NVDA Open,NVDA High,NVDA Low,NVDA Close,NVDA Volume,NVDA Daily change,NVDA Daily change Max,AAPL Open,AAPL High,...,MSFT Volume,MSFT Daily change,MSFT Daily change Max,INTC Open,INTC High,INTC Low,INTC Close,INTC Volume,INTC Daily change,INTC Daily change Max
2495,"Jul 25, 2012",3.20,3.31,3.18,3.27,41504400,2.187500,4.088050,20.52,20.74,...,45579500,-1.402189,1.911049,25.02,25.53,25.00,25.13,34407600,0.439648,2.120000
2496,"Jul 24, 2012",3.25,3.27,3.17,3.21,35116400,-1.230769,3.154574,21.69,21.77,...,47723300,-0.307798,1.591696,25.19,25.22,24.79,25.01,33637600,-0.714569,1.734570
2497,"Jul 23, 2012",3.12,3.27,3.08,3.24,45158400,3.846154,6.168831,21.23,21.64,...,55151900,-0.980724,1.964840,25.05,25.36,24.80,25.26,36378200,0.838323,2.258065
2498,"Jul 20, 2012",3.28,3.30,3.17,3.20,45100000,-2.439024,4.100946,21.89,21.94,...,64021700,-2.838710,3.327787,25.87,26.00,25.50,25.52,48844000,-1.352918,1.960784
2499,"Jul 19, 2012",3.29,3.34,3.27,3.30,40303600,0.303951,2.140673,21.83,21.98,...,46663200,0.524418,1.382488,26.35,26.35,25.82,26.06,36555800,-1.100569,2.052672
